<a href="https://colab.research.google.com/github/limdiny/ML/blob/main/%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블
* 앙상블 학습

더 좋은 예측 결과를 만들기 위해 여러개의 모델을 훈련하는 머신러닝 알고리즘이다.

* 랜덤 포레스트

대표적인 결정 트리 기반의 앙상블 학습 방법이다.부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만드는 것이 특징이다.

* 엑스트라 트리

랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않는다. 대신 랜덤하게 노드를 분할해 과소적합을 감소시킨다.

* 그레이디언트 부스팅

랜덤 포레스트와 엑스트라 트리와 달리 결정트리를 연속으로 추가하여 손실함수를 최소화하는 앙상블 방법이다. 훈련 속도는 느리지만 더 좋은 성능을 기대할 수 있다.
그레이디언트 부스팅의 속도를 개선한 것이 **히스토그램 기반 그레이디언트 부스팅** 이며 안정적인 결과와 높은 성능으로 인기가 높다.

* RandomForestClassifier

* ExtraTreesClassifier

* GrandientBoostingClassifier

* HistGrandientBoostingClassifier

# 랜덤 포레스트

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier(n_jobs=-1,random_state=42)

scores=cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


* 랜덤포레스트를 교차검증해줬음
* return_train_score=True로 지정하면 훈련점수도 같이 나옴
* 과대적합

In [ ]:
rf.fit(train_input,train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


* 특성중요도
* [알코올,당도,pH]
* 앞절 결정트리에서 당도의 특성중요도가 거의 86% 였는데 다른 결과 --> 랜덤하게 특성을 뽑아서 훈련시키기때문이다. 

In [ ]:
rf=RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


* 부트스트랩 샘플에 포함되지 않는 샘플들 : OOB샘플
* 남는 샘플로 평가할 수 있음( 마치 검증세트의 역할)

# 엑스트라 트리

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


* 랜덤 포레스트와 비슷하지만 부트스트랩 샘플을 사용하지않고 모든 샘플을 사용함
* 노드를 분할할때 중요도로 나누는 것이 아니라 무작위로 노드를 나눔
* 엑스트라 트리가 무작위성이 더 크기때문에 결정트리가 더 많이 필요하지만 노드를 무작위로 분할하기때문에 빠른 속도임

In [ ]:
et.fit(train_input,train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그레이디언트 부스팅

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


* 그레이디언트 부스팅은 얕은 깊이의 결정 트리를 사용해서 이전 트리의 오차를 보완하는 방식으로 앙상블을 한다.
* 깊이가 얕아서 과대적합에 강하고 높은 일반화 성능을 기대할 수 있다.
* 경사하강법을 사용하여 트리를 앙상블에 추가한다.

In [ ]:
gb=GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


* 기본: 깊이 3, 트리 100개, 학습률 0.1 ----> 트리 500개, 학습률 0.2
* 트리를 5배나 늘렸는데도 과대적합을 잘 억제함

In [ ]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# 히스토그램 기반 그레이디언트 부스팅

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb,train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


0.9321723946453317 0.8801241948619236


* 입력 특성을 256개의 구간으로 나누어 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.
* 결과를 봤을때 그레이디언트 부스팅보다 조금 더 높은 성능을 제공한다

In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input,train_target)
result=permutation_importance(hgb,train_input,train_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


* permutation_importance는  특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변하는지 관찰을 해서 특성 중요도를 계산한다. 

* n_repeats는 랜덤하게 특성을 섞을 횟수를 지정한다.

* permutation_importance 가 반환하는 객체는 반복하여 얻은 특성중요도,평균,표준편차이다.

In [ ]:
result=permutation_importance(hgb,test_input,test_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)
hgb.score(test_input,test_target)

[0.05969231 0.20238462 0.049     ]


0.8723076923076923